In [4]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [36]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#experiments with dask
import dask.dataframe as dd
import dask_geopandas as dg

container = 'ana-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/ana_station_metatdata.csv'
station_df = pd.read_csv(station_url, storage_options=storage_options)                
df = dd.read_csv(station_url, storage_options=storage_options)
#ddf = dg.from_dask_dataframe(df).set_geometry(dg.points_from_xy(ddf, 'Latitude', 'Longitude'))

"\nstation_list = pd.DataFrame({'site_no':filter(None,                           map(lambda sub:(''.join([ele for ele in sub if ele.isnumeric()])), fs_list))}).astype('int')\nfiltered_station_df = station_list.merge(station_df, on='site_no')\n#filter the stations to ones with data\n\ncrs = {'init': 'epsg:4326'}\ngdf = gpd.GeoDataFrame(filtered_station_df,                       geometry=gpd.points_from_xy(filtered_station_df.Longitude, filtered_station_df.Latitude))\nbuffer_style = {'round':1,'flat':2,'square':3}\nbuffer = 0.025\ngdf.geometry = gdf.geometry.buffer(buffer, cap_style = buffer_style['square'], resolution=1)\n\ngdf = gdf.drop_duplicates()\n"

# Here we preprocessing of the data for standardized ingestion

In [184]:
for site_no in station_df.site_no:
    station_url = f'az://{container}/{site_no}.csv'
    station_url2 = f'az://{container}/{site_no}_2.csv'
    site_df1_raw = pd.read_csv(station_url, delimiter=';', skiprows=10, storage_options=storage_options)
    translation = pd.read_csv(f'az://{container}/ana_translations.csv', storage_options=storage_options)
    trans = {p:e  for p,e in zip(translation.Portuguese, translation.English)}
    site_df1 = site_df1_raw.rename(columns=trans)
    site_df1 = site_df1.dropna(subset=['Date'])
    site_df1['TimeL'] = site_df1['TimeL'].fillna('01/01/1900 01:00')
    site_df1['Date-Time'] = [f"{d} {t.split(' ')[1]}" for d,t in zip(site_df1['Date'],site_df1['TimeL'])]
    site_df1['Date-Time'] = pd.to_datetime(site_df1['Date-Time'])

    site_df2_raw = pd.read_csv(station_url2, delimiter=';', skiprows=14, storage_options=storage_options)
    site_df2_raw = site_df2_raw.replace('01/01/1900', '01/01/1900 01:00')
    translation2 = {'Data':'Date','Hora':'Hour','Turbidez':'Turbidity'}
    site_df2 = site_df2_raw.rename(columns=translation2)
    site_df2 = site_df2.dropna(subset=['Date'])
    site_df2['Date-Time'] = [f"{d} {t.split(' ')[1]}" for d,t in zip(site_df2['Date'],site_df2['Hour'])]
    site_df2['Date-Time'] = pd.to_datetime(site_df2['Date-Time'])
    site_df2 = site_df2[['Date', 'Hour', 'Date-Time','Turbidity']]

    selection = ['Date-Time', 'Discharge', 'Suspended Sediment Concentration (mg/L)', 'Turbidity']
    site_df = site_df1.merge(site_df2, on='Date', how='outer', suffixes=('_',''))
    site_df['Date-Time'] = site_df['Date-Time'].fillna(site_df['Date-Time_'])
    #site_df['Hour'] = site_df['Hour'].fillna(site_df['Hour_'])
    site_df = site_df[selection]

    write_filename = f'az://{container}/stations/{site_no}.csv'
    site_df.to_csv(write_filename, index=False, storage_options=storage_options)

21750000
22050001
22500000
22700000
22900000
23100000
23230000
23250000
23700000
25200000
26200000
26300000
26350000
26800000
27500000
27550000
28240000
28300000
28850000
29050000
29100000
29200000
